In [0]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import librosa
import math

%matplotlib inline

### Helper functions

In [2]:
# In Google Colaborary
from google.colab import files
train_clean_male_wav = files.upload() # train_clean_male.wav
train_dirty_male_wav = files.upload() # train_clean_male.wav




Saving train_clean_male.wav to train_clean_male.wav


Saving train_dirty_male.wav to train_dirty_male.wav


In [3]:
# In Google Colaborary
from google.colab import files
test_x_01 = files.upload() # train_clean_male.wav
test_x_02 = files.upload() # train_clean_male.wav




Saving test_x_01.wav to test_x_01.wav


Saving test_x_02.wav to test_x_02.wav


In [0]:
# Calculate SNR 
def calculateSNR(st, st_h):
    st_sum = np.sum(np.abs(st))
    diff_sum = np.sum(np.abs(st-st_h))
    return 10*math.log(st_sum**2/diff_sum**2)

In [5]:
# Read training data
s, sr_s=librosa.load('train_clean_male.wav', sr=None)
S=librosa.stft(s, n_fft=1024, hop_length=512)
sn, sr_x=librosa.load('train_dirty_male.wav', sr=None)
X=librosa.stft(sn, n_fft=1024, hop_length=512)
print(S.shape)
print(X.shape)
S_abs = np.abs(S)
X_abs = np.abs(X)

(513, 2459)
(513, 2459)


In [0]:
is_training = True
def get_cnn1_model():
    x = tf.placeholder(tf.float32, [None, 513, 1])
    y_ = tf.placeholder(tf.float32, [None, 513])
    LEARNING_RATE = 10e-4
    BATCH_SIZE = 32
    
    initializer = tf.contrib.layers.variance_scaling_initializer()

    conv1 = tf.layers.conv1d(
        x,            
        filters=32,
        kernel_size=3,
        activation=tf.nn.relu)

    conv2 = tf.layers.conv1d(
        conv1,            
        filters=64,
        kernel_size=3,
        activation=tf.nn.relu)

    conv3 = tf.layers.conv1d(
        conv2,            
        filters=128,
        kernel_size=3,
        activation=tf.nn.relu)

    flat = tf.reshape(conv3, [-1, 128*507])
    ln1 = tf.contrib.layers.layer_norm(flat)
    l2 = tf.layers.dense(ln1, units=1024,  activation=tf.nn.relu, kernel_initializer=initializer)
    dropout1 = tf.layers.dropout(l2, rate=0.2, training=is_training)
    ln2 = tf.contrib.layers.layer_norm(dropout1)
    l3 = tf.layers.dense(ln2, units=1024,  activation=tf.nn.relu, kernel_initializer=initializer)
    dropout2 = tf.layers.dropout(l3, rate=0.2, training=is_training)
    y = tf.layers.dense(dropout2, units=513, activation=tf.nn.relu, kernel_initializer=initializer)

    loss = tf.losses.mean_squared_error(labels=y_, predictions=y)
    train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)

    sess=tf.Session()
    tf.global_variables_initializer().run(session=sess)

    train_data = tf.data.Dataset.from_tensor_slices(tf.constant(X_abs.T.reshape(-1, 513, 1)))
    label_data = tf.data.Dataset.from_tensor_slices(tf.constant(S_abs.T))
    batch_data = tf.data.Dataset.zip((train_data, label_data)).repeat().batch(BATCH_SIZE)

    iterator = batch_data.make_one_shot_iterator()
    next_batch = iterator.get_next()
    return (x, y_, y, sess, next_batch, train_step, loss)



In [31]:
(x1, y_1, y1, sess_1, next_batch_1, train_step_1, loss_1) = get_cnn1_model()

saver = tf.train.Saver()

TRAIN_STEPS = 10000


for i in range(TRAIN_STEPS+1):    
    batch = sess_1.run(next_batch_1)
    _, loss_value = sess_1.run((train_step_1, loss_1), feed_dict={x1: batch[0], y_1: batch[1]})
    if i% 500 == 0:
        print('Training Step:' + str(i) + '  Loss =  ' + str(loss_value))
        


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Training Step:0  Loss =  1.34463
Training Step:500  Loss =  0.028354645
Training Step:1000  Loss =  0.04534659
Training Step:1500  Loss =  0.0008305101
Training Step:2000  Loss =  0.020449989
Training Step:2500  Loss =  0.03604931
Training Step:3000  Loss =  0.016032385
Training Step:3500  Loss =  0.029099604
Training Step:4000  Loss =  0.012278512
Training Step:4500  Loss =  0.017634071
Training Step:5000  Loss =  0.0061632036
Training Step:5500  Loss =  0.067448474
Training Step:6000  Loss =  0.015156337
Training Step:6500  Loss =  0.017445985
Training Step:7000  Loss =  0.005189274
Training Step:7500  Loss =  0.016518248
Training Step:8000  Loss =  0.008710186
Training Step:8500  Loss =  0.003125579
Training Step:9000  Loss =  0.0052581294
Training St

In [23]:
# Calculate SNR by training data
is_training = False
S_test_abs = sess_1.run(y1, feed_dict={x1: X_abs.T.reshape(-1, 513, 1) }).T
is_training = True
ratio = (X / X_abs)
s_t = np.multiply(ratio, S_test_abs)
print(calculateSNR(S, s_t))

15.089389977662648


In [0]:
def denoise_sound_conv1(sess, x, y, input_file_name, output_file_name):
    sn, sr=librosa.load(input_file_name, sr=None)
    testX=librosa.stft(sn, n_fft=1024, hop_length=512)
    testX_abs = np.abs(testX)
    is_training = False
    S_test_abs = sess.run(y, feed_dict={x: testX_abs.T.reshape(-1, 513, 1) }).T
    is_training = True
    print(S_test_abs.shape)
    ratio = (testX / testX_abs)
    Sh = np.multiply(ratio, S_test_abs)
    librosa.output.write_wav(output_file_name, librosa.istft(Sh, hop_length=512), sr)
    

In [25]:
denoise_sound_conv1(sess_1, x1, y1, 'test_x_01.wav', 'recover_01_d1x.wav')
denoise_sound_conv1(sess_1, x1, y1, 'test_x_02.wav', 'recover_02_d1x.wav')

(513, 142)
(513, 380)


In [0]:
files.download('recover_01_d1x.wav')
files.download('recover_02_d1x.wav')


### CNN 2D


In [0]:
# Transform a spectra to image like data form with dim 20x513
def imageize_spec(X):
    raw = []
    for t in range(len(X)-19):
        raw.append(X[t: t+20])
    X_abs_2d = np.array(raw).reshape(-1, 20, 513, 1)
    return X_abs_2d    

In [0]:
# Padding 
def padding_frames(X):
    testX_pad = X
    # padding
    for i in range(19):
        testX_pad = np.hstack((testX_pad, np.zeros((513, 1))))
    return testX_pad


In [9]:
# "imageize" data for 2D CNN, before that we will pad 19 empty frame at the end
X_abs_T = np.abs(padding_frames(X)).T
X_abs_2d = imageize_spec(X_abs_T)

print(X_abs_2d.shape)

(2459, 20, 513, 1)


In [0]:
is_training = True
def get_cnn2_model():
    x = tf.placeholder(tf.float32, [None, 20, 513, 1])
    y_ = tf.placeholder(tf.float32, [None, 513])
    LEARNING_RATE = 10e-5
    BATCH_SIZE = 64
    
    initializer = tf.contrib.layers.variance_scaling_initializer()

    #  18 * 511 * 16 
    conv1 = tf.layers.conv2d(
        x,            
        filters=16,
        kernel_size=[3, 3],
        activation=tf.nn.relu)

    # 9 * 255 * 16
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # 7 * 253 * 32
    conv2 = tf.layers.conv2d(
        pool1,            
        filters=64,
        kernel_size=[3, 3],
        activation=tf.nn.relu)

    # 3 * 126 * 32
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # 1 * 124 * 64
    conv3 = tf.layers.conv2d(
        pool2,            
        filters=64,
        kernel_size=[3, 3],
        activation=tf.nn.relu)

    flat = tf.reshape(conv3, [-1, 124*64])
    
    ln1 = tf.contrib.layers.layer_norm(flat)
    l2 = tf.layers.dense(ln1, units=1024,  activation=tf.nn.relu, kernel_initializer=initializer)
    dropout1 = tf.layers.dropout(l2, rate=0.2, training=is_training)
    ln2 = tf.contrib.layers.layer_norm(dropout1)
    l3 = tf.layers.dense(ln2, units=1024,  activation=tf.nn.relu, kernel_initializer=initializer)
    dropout2 = tf.layers.dropout(l3, rate=0.2, training=is_training)
    y = tf.layers.dense(dropout2, units=513, activation=tf.nn.relu, kernel_initializer=initializer)

    loss = tf.losses.mean_squared_error(labels=y_, predictions=y)
    train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)

    sess=tf.Session()
    tf.global_variables_initializer().run(session=sess)
    
    # Modify data for training
    train_data = tf.data.Dataset.from_tensor_slices(tf.constant(X_abs_2d))
    label_data = tf.data.Dataset.from_tensor_slices(tf.constant(S_abs.T))
    batch_data = tf.data.Dataset.zip((train_data, label_data)).repeat().batch(BATCH_SIZE)

    iterator = batch_data.make_one_shot_iterator()
    next_batch = iterator.get_next()

    return (x, y_, y, sess, next_batch, train_step, loss)



In [16]:
(x2, y_2, y2, sess_2, next_batch_2, train_step_2, loss_2) = get_cnn2_model()

TRAIN_STEPS = 15000

for i in range(TRAIN_STEPS+1):    
    batch = sess_2.run(next_batch_2)
    _, loss_value = sess_2.run((train_step_2, loss_2), feed_dict={x2: batch[0], y_2: batch[1]})
    if i% 500 == 0:
        print('Training Step:' + str(i) + '  Loss =  ' + str(loss_value))


Training Step:0  Loss =  1.2867198
Training Step:500  Loss =  0.0798838
Training Step:1000  Loss =  0.035159424
Training Step:1500  Loss =  0.025857322
Training Step:2000  Loss =  0.014877802
Training Step:2500  Loss =  0.016420899
Training Step:3000  Loss =  0.014716226
Training Step:3500  Loss =  0.013950449
Training Step:4000  Loss =  0.013395712
Training Step:4500  Loss =  0.01250609
Training Step:5000  Loss =  0.01670644
Training Step:5500  Loss =  0.011480769
Training Step:6000  Loss =  0.0070692473
Training Step:6500  Loss =  0.008225092
Training Step:7000  Loss =  0.0051851915
Training Step:7500  Loss =  0.00489032
Training Step:8000  Loss =  0.0042150742
Training Step:8500  Loss =  0.0019531136
Training Step:9000  Loss =  0.0057302997
Training Step:9500  Loss =  0.009187471
Training Step:10000  Loss =  0.008794038
Training Step:10500  Loss =  0.012084844
Training Step:11000  Loss =  0.0075779054
Training Step:11500  Loss =  0.004853153
Training Step:12000  Loss =  0.004122775


In [17]:
# Calculate SNR by training data

X_abs_T = np.abs(padding_frames(X)).T
X_abs_2d = imageize_spec(X_abs_T)

is_training = False
S_test_abs = sess_2.run(y2, feed_dict={x2: X_abs_2d }).T
is_training = True

ratio = (X / X_abs)
s_t = np.multiply(ratio, S_test_abs)
print(calculateSNR(S, s_t))


13.836223441561268


In [0]:
def denoise_sound_conv2(sess, x, y, input_file_name, output_file_name):
    sn, sr=librosa.load(input_file_name, sr=None)
    testX=librosa.stft(sn, n_fft=1024, hop_length=512)
    testX_abs = np.abs(testX)
    testX_abs_pad = np.abs(padding_frames(testX))
    X_abs_2d = imageize_spec(testX_abs_pad.T)
    is_training = False
    S_test_abs = sess.run(y, feed_dict={x: X_abs_2d }).T
    is_training = True
    print(S_test_abs.shape)
    ratio = (testX / testX_abs)
    Sh = np.multiply(ratio, S_test_abs)
    librosa.output.write_wav(output_file_name, librosa.istft(Sh, hop_length=512), sr)
    

In [19]:
denoise_sound_conv2(sess_2, x2, y2, 'test_x_01.wav', 'recover_01_d2x.wav')
denoise_sound_conv2(sess_2, x2, y2, 'test_x_02.wav', 'recover_02_d2x.wav')

(513, 142)
(513, 380)


In [0]:
files.download('recover_01_d2x.wav')
files.download('recover_02_d2x.wav')


## Result

I found the training time of 1D CNN is much faster than 2D CNN, since the parameters to train is much smaller in 1D CNN model.

For SNR for the training data, 1D CNN model can reached 15dB, and 2D CNN can also reach 1dB.

However, even the SNR of 2D CNN can be as good as the 1D CNN, I still feel the quality of 1D CNN is much better than 2D CNN based on my subjective listening test.